In [30]:
import pandas
import numpy as np
import csv
%matplotlib inline
import matplotlib.pyplot as plot
import sklearn.linear_model
from sklearn.impute import KNNImputer
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler

In [132]:
#import and read files
df_cleveland = pandas.read_csv("processed.cleveland_H.data", sep=",",encoding='unicode_escape', header=None)
df_hungarian = pandas.read_csv("processed.hungarian_H.data", sep=",",encoding='unicode_escape', header=None)
df_switzerland = pandas.read_csv("processed.switzerland_H.data", sep=",",encoding='unicode_escape', header=None)
df_va = pandas.read_csv("processed.va_H.data", sep=",",encoding='unicode_escape', header=None)

In [133]:
#create dataframe
df = [df_cleveland, df_hungarian, df_switzerland, df_va]
full_df = pandas.concat(df)
full_df.columns=['age','sex', 'chest pain type', 'resting bp(mm/Hg)', 
                 'serum cholesterol(mg/dl)', 'fasting bs', 'resting ecg',
                'max hr', 'ex induced angina', 'ST depression', 'slope',
                'nmv', 'thal', 'diagnosed prediction']

#we need the data as x and y where x is the data set and y is the target value
#For y, we take the target value or diagnosed predictions so that any value greater than
#0, is assigned 1. This means if there was no presence of heart disease, then
#0 is assigned, and any presence (1-4) is assigned 1. This will help scale the 
#our predictions as this is a binary classification approach.

#clean data
#convert all data if numerical to float(except diagnoses prediction)
#if data isn't numerical, make it NaN
full_df = full_df.apply(pandas.to_numeric, errors='coerce')

x = full_df.drop('diagnosed prediction', axis=1)
y = full_df.drop(['age','sex', 'chest pain type', 'resting bp(mm/Hg)', 
                 'serum cholesterol(mg/dl)', 'fasting bs', 'resting ecg',
                'max hr', 'ex induced angina', 'ST depression', 'slope',
                'nmv', 'thal'], axis=1)
y['diagnosed prediction'].replace([1,2,3,4],[1,1,1,1],inplace=True)
y_array = y.to_numpy()


In [134]:
#We plan on having 6 different cases. To avoid a data leak, the plan is to
#split the data into 70/15/15 as per literaure. Or 70% for training, 15% for validation, 
#15% for testing. From here, we will then impute and scale each set of data.
#In total there will be 6x3=18 data sets. 6 will be trained, 6 will be used for
#validation, and 6 will be used for testing.

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y_array, test_size=0.3, random_state=50)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=50)

x_train



,age,sex,chest pain type,resting bp(mm/Hg),serum cholesterol(mg/dl),fasting bs,resting ecg,max hr,ex induced angina,ST depression,slope,nmv,thal
60,43.0,0.0,2.0,120.0,201.0,0.0,0.0,165.0,0.0,0.0,NaN,NaN,NaN
178,59.0,0.0,2.0,130.0,188.0,0.0,0.0,124.0,0.0,1.0,2.0,NaN,NaN
53,55.0,1.0,4.0,120.0,0.0,0.0,1.0,92.0,0.0,0.3,1.0,NaN,7.0
180,59.0,1.0,3.0,130.0,318.0,0.0,0.0,120.0,1.0,1.0,2.0,NaN,3.0
52,42.0,0.0,3.0,115.0,211.0,0.0,1.0,137.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,59.0,1.0,4.0,140.0,177.0,0.0,0.0,162.0,1.0,0.0,1.0,1.0,7.0
270,61.0,1.0,4.0,140.0,207.0,0.0,2.0,138.0,1.0,1.9,1.0,1.0,7.0
140,75.0,1.0,4.0,160.0,310.0,1.0,0.0,112.0,1.0,2.0,3.0,NaN,7.0
132,53.0,0.0,2.0,140.0,216.0,0.0,0.0,142.0,1.0,2.0,2.0,NaN,NaN


In [135]:
#An issue with the data is that much of it, depending on its source
#contains missing values denoted as "?". We took those values and
#replaced them with "NaN". Now, we're going to look at multiple
#cases where we attempt to find ways to work around these missing values

#First, let's create a scaling function

#Here we created a function named scaled which takes in x, a dataframe.
#It will then scaled all the data between 0-1 as this works best for neural networks.
#We will assign this function to every dataframe we intend on using (training/testing).
def scale(x):
    minimax_scaler = preprocessing.MinMaxScaler()
    scaled = pandas.DataFrame(minimax_scaler.fit_transform(x.values), columns=x.columns, 
                              index=x.index)
    return scaled

#Case 1: Now that we have separated the data, we are going to impute it.
#Our first imputition technique will setting all NaN values to 0.
#We are also going to scale the data between 0-1. This range works best 
#for neural networks
df1_train_data = x_train.fillna(0)
df1_test_data = x_test.fillna(0)
df1_val_data = x_val.fillna(0)

#Now, let's scale each set of data.
df1_train_scaled= scale(df1_train_data)
df1_test_scaled= scale(df1_test_data)
df1_val_scaled= scale(df1_val_data)

#Now that we have split, imputed, and scaled the data. Let's do the same for the rest of
#our cases


In [136]:
#case 6: Here we are going to use k-NearestNeighbor(kNN) imputation, which is 
#another PVI to predict the missing values of each column. 
#We are going to use kNN on the data set with a k value set to 4.

#first let's create a kNN imputation function
def kNNimpute(x):
    kNNimputer = KNNImputer(n_neighbors=4)
    imputed = pandas.DataFrame(kNNimputer.fit_transform(x), columns=x.columns, index=x.index)
    
    return imputed

df6_train_data = kNNimpute(x_train)
df6_test_data = kNNimpute(x_test)
df6_val_data = kNNimpute(x_val)

#scale
df6_train_scaled= scale(df6_train_data)
df6_test_scaled= scale(df6_test_data)
df6_val_scaled= scale(df6_val_data)


In [137]:
#Now that our data has been split into train, test, and validate into
#ratios of 70/15/15 respectively, we can begin building our model

#The layout for the neural network is as follows:

#Forward Pass:

#Our input layer will be all scaled x values. They will be assigned 
#random weights which will be multiplied by each x.
#All x's at every neuron in the first dense (hidden) layer will 
#be summed and have a bias added to them. From there, we now have
#output values from the input layer in the neurons of the 
#first hidden layer. Because we summed different weights multiplied
#by x, and then added them to a bias. Each input in the first hidden
#layer will contain a different value. Here, we will use our first 
#activation function. With the goal of creating the best model for
#this problem in mind, we are going to use two commonly chosen 
#acitvation functions for this layer: ReLU and Leaky ReLU.

#ReLU will return either 0, if the value produced is less than or 
#equal to 0, or it will return the value itself if it greater than 0.

#Leaky ReLU is used to combat the dying ReLU problem. This occurs when
#many of the outputs of ReLU are negative, creating a bias. It can
#also occur when the learning rate is too high. Leaky ReLU will return the
#value itself if it's greater than 0. If it is equal to 0, it will return
#0, and if it is less than 0, the slope of the function will tell us that
#it will return a number barely, but increasingly smaller than 0. 
#This is because the slope is very small, but not zero. 

#For the second hidden layer, we wil use the Sigmoid function. This will
#allow us to return probabilities in the output layer.

#Loss:

#For loss, we will use cross-entropy loss function. This will be applied 
#to the second hidden layer with respect to the predictions, y.
#We will also use L2 or ridge regularization to calculate the loss on 
#both hidden layers. This is the loss calculated from our optimizer.
#The total amount of loss will be the product of both types of loss.

#Backpropogation:

#For backpropogation, we need to apply the chain rule to every layer.
#For this, we first start with the derivative of the BinaryCrossEntropy
#with respect to the sigmoid activation output. Then, we take the 
#derivative of the sigmoid function with respect to the derivative
#of the inputs of BinaryCrossEntropy. Next, we take the derivative
#of the values of the input of sigmoid function. Then we take the 
#derivative of ReLU with respect to the derivative of the inputs 
#of the second hidden layer. We will then take the derivative of 
#the values and L2 regularization of weights and biases with respect to
#the input of the derivative of ReLU.

#We will then update our hyperparameters which are the weights and biases
#along with the learning rate from our optimizer, Adam. This will be done
#until we recieve an acceptable loss on our training, validation, and 
#testing data.


In [402]:
class Hidden_Layer:
    
    #initialize the layer
    def __init__(self, neuron_inputs, neuron_neurons, L2_weight=0, L2_bias=0):
        #initalize the variables
        
        #randomize the weights from a Gaussian Distribution with 0 as the
        #mean and 1 as the variance
        self.weights = 0.01 * np.random.randn(neuron_inputs, neuron_neurons)
        self.biases = np.zeros((1, neuron_neurons))
        
        #initialize regularization for weight and bias
        
        self.L2_weight = L2_weight
        self.L2_bias = L2_bias
       
        
    #forward pass function for the first hidden layer                                  
    def forward_pass(self, inputs):
        
        #saving the inputs for backwards pass
        #the inputs of the forward pass is the input (data) itself
        self.inputs = inputs
        
        #the output is the dot product of the inputs and weights 
        #plus the bias
        self.outputs = np.dot(inputs, self.weights) + self.biases
        
    def backward_pass(self, dvalues):
        
        #calculate the gradients of weights, biases, and inputs
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        
        
        #calculate the gradient on L2 regularization (derivative  of L2)
        
        if self.L2_weight > 0:
            self.dweights += 2*self.L2_weight*self.weights
            
        if self.L2_bias > 0:
            self.dbiases += 2*self.L2_bias*self.biases
        
        self.dinputs = np.dot(dvalues, self.weights.T)


In [403]:
class ReLU:
    #forward pass using the output from the hidden layer as it's input
    def forward_pass(self, inputs):
        
        self.inputs = inputs
    
        #the ReLU activation function will return 0 if the input is smaller
        #than 0, and the actual input if greater than 0
        self.outputs = np.maximum(0, inputs)
    
    def backward_pass(self, dvalues):
        self.dinputs = dvalues.copy()
        
        #all derivatives greater than 0 are equal to themselves
        self.dinputs[self.inputs > 0 ] = self.dinputs[self.inputs>0]
        
        #all derivatives less than 0 are equal to 0
        self.dinputs[self.inputs <= 0] = 0
        
        


In [404]:
class Sigmoid:
    def predictions(self, outputs):
        return np.round(outputs)
    
    def forward_pass(self, inputs):
        #self.inputs = inputs
        self.outputs = 1 / (1+np.exp(-inputs))
        
    def backward_pass(self, dvalues):
        self.dinputs = dvalues * (1-self.outputs) * self.outputs
       

In [405]:
class Loss:
    def L2reg_loss(self, layer):
        #initialize loss to 0
        L2reg_loss = 0
        
        #changing the regularization loss if the weights 
        if layer.L2_weight >0:
            L2reg_loss += layer.L2_weight* np.sum(layer.weights * layer.weights)
        if layer.L2_bias >0:
            L2reg_loss += layer.L2_bias*np.sum(layer.biases * layer.biases)
                
        return L2reg_loss
                                              
    
    def calculate(self, outputs, y):
        sample_loss = self.forward_pass(outputs, y)
        data_loss = np.mean(sample_loss)
        return data_loss

    
class BinaryCrossEntropy(Loss):
    def forward_pass(self, predicted_y, true_y):
        predicted_y_clipped = np.clip(predicted_y, 1e-7, 1-1e-7)
        sample_loss = -(true_y * np.log(predicted_y_clipped)+
                        (1-true_y) * np.log(1-predicted_y_clipped))
        sample_loss = np.mean(sample_loss, axis=-1)
        return sample_loss
    
    def backward_pass(self, dvalues, true_y):
        dvalues_clipped = np.clip(dvalues, 1e-7, 1-1e-7)
        num_samples = len(dvalues)
        num_outputs = len(dvalues[0])
        
        self.dinputs = -(true_y / dvalues_clipped - (1-true_y) / \
                         (1-dvalues_clipped)) / num_outputs
        self.dinputs = self.dinputs / num_samples

        

In [406]:
#class for optimizer. In this case we're using Adam
class Adam:
    def __init__(self, learning_rate=0, decay=0, epsilon=0.00001, b1=0.9, b2=0.999):
        self.learning_rate = learning_rate 
        self.current_learning_rate = learning_rate 
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon 
        self.b1 = b1 
        self.b2 = b2
        
    #before we update parameters 
    def pre_parameter_update(self): 
        if self.decay:
            self.current_learning_rate = self.learning_rate * \
                                        (1. / (1. + self.decay * self.iterations))
    def parameter_update(self, layer):
        
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentum = np.zeros_like(layer.weights) 
            layer.weight_cache = np.zeros_like(layer.weights) 
            layer.bias_momentum = np.zeros_like(layer.biases) 
            layer.bias_cache = np.zeros_like(layer.biases)
        
        
        #calculate momentum for the bias and weight using beta 1
        layer.weight_momentum = self.b1 *  layer.weight_momentum + \
                                (1 - self.b1) * layer.dweights 
        layer.bias_momentum = self.b1 * layer.bias_momentum + \
                                (1 - self.b1) * layer.dbiases
        
        #calculate cache for weight and bias using beta 2 (same as RMS prop)
        layer.weight_cache = self.b2 * layer.weight_cache + \
                                (1 - self.b2) * (layer.dweights**2)
        layer.bias_cache = self.b2 * layer.bias_cache + \
                                (1 - self.b2) * (layer.dbiases**2)
    
        #get corrected weight and bias momentum
        weight_momentum_corrected = layer.weight_momentum / \
                                (1 - self.b1 ** (self.iterations + 1))
        bias_momentum_corrected = layer.bias_momentum / \
                                (1 - self.b1 ** (self.iterations + 1))
        
        #get corrected weight and bias cache
        weight_cache_corrected = layer.weight_cache / \
                                (1 - self.b2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / \
                                (1 - self.b2 ** (self.iterations + 1))
        
        #now we update the weights and bias
        layer.weights += -self.current_learning_rate * \
                                weight_momentum_corrected / (np.sqrt(weight_cache_corrected) +
                                                             self.epsilon)
        layer.biases += -self.current_learning_rate * \
                                bias_momentum_corrected / (np.sqrt(bias_cache_corrected) +
                                                           self.epsilon)                 
        
    def post_parameter_update(self): 
        self.iterations = self.iterations + 1
            

In [407]:
class Dropout:
    def __init__(self, rate):
        self.rate = 1-rate
    def forward_pass(self, inputs):
        self.inputs = inputs
        self.binary_mask = np.random.binomial(1, self.rate, size=inputs.shape) / self.rate
        self.outputs = inputs * self.binary_mask
    def backward_pass(self, dvalues):
        self.dinputs = dvalues * self.binary_mask

In [409]:
#create a hidden layer. Given 13 features we will produce 14 outputs(neurons)
first_hidden = Hidden_Layer(13, 340, L2_weight=0.001, L2_bias=0.001)

y_train = y_train.reshape(-1,1)
#print(y_train)

#initialize ReLU activation
ReLU_activation = ReLU()

first_dropout = Dropout(0.15)

#create a second hidden layer
second_hidden = Hidden_Layer(340, 1)

#initialize sigmoid
Sigmoid_activation = Sigmoid()

#initialize loss function
BinaryCrossEntropyloss = BinaryCrossEntropy()

#initialize optimizing function
Adam_optimizer = Adam(learning_rate=0.0003, decay=0.00001)



for epoch in range(220):

        #we perform the forward pass on the training data
        first_hidden.forward_pass(df6_train_scaled)

        #forward pass the outputs of the first layer to ReLU
        ReLU_activation.forward_pass(first_hidden.outputs)
        
        first_dropout.forward_pass(ReLU_activation.outputs)

        #forward pass the outputs of ReLU in the second layer
        second_hidden.forward_pass(first_dropout.outputs)
        
        #forward pass the outputs of the second hidden layer to sigmoid
        Sigmoid_activation.forward_pass(second_hidden.outputs)
        
        #calculate loss on forward pass
        loss = BinaryCrossEntropyloss.calculate(Sigmoid_activation.outputs, y_train)
        
        #calculate regularization loss
    
        
        L2reg_loss = \
            BinaryCrossEntropyloss.L2reg_loss(first_hidden) + BinaryCrossEntropyloss.L2reg_loss(second_hidden)
        total_loss = loss + L2reg_loss
        
        
        
        #compare outputs from softmax to y for accuracy
        probabilities =  Sigmoid_activation.outputs
        predictions = np.round(probabilities)
        
        #predictions = (Sigmoid_activation.outputs > 0.5) *  1
        accuracy = np.mean(predictions == y_train)
        
        if not epoch % 5: 
            print(f'epoch: {epoch}, ' +
                  f'acc: {accuracy:.3f}, ' +
                  f'loss: {loss:.3f}, ' +
                  f'data_loss: {loss:.3f}, ' +
                  f'reg_loss: {L2reg_loss:.3f}, ' +
                  f'lr: {Adam_optimizer.current_learning_rate}')
            
        #THE CHAIN RULE APPLIED HERE!!!

        #backwards pass the loss_activation output through the loss activation backward
        BinaryCrossEntropyloss.backward_pass(Sigmoid_activation.outputs, y_train)

        Sigmoid_activation.backward_pass(BinaryCrossEntropyloss.dinputs)

        #backwards pass the derivative of the loss activation inputs through
        #the backward function of dense2
        second_hidden.backward_pass(Sigmoid_activation.dinputs)
        
        first_dropout.backward_pass(second_hidden.dinputs)


        #backwards pass the derivative of the dense2 derivative inputs
        #through the backward function of first activation function
        ReLU_activation.backward_pass(first_dropout.dinputs)
     
        #backwards pass the derivative of the first activation function inputs
        #through the backwards function of dense1
        first_hidden.backward_pass(ReLU_activation.dinputs)
            
        
        Adam_optimizer.pre_parameter_update()
        Adam_optimizer.parameter_update(first_hidden)
        Adam_optimizer.parameter_update(second_hidden) 
        Adam_optimizer.post_parameter_update()
        

y_val = y_val.reshape(-1,1) 

first_hidden.forward_pass(df6_val_scaled)

ReLU_activation.forward_pass(first_hidden.outputs)

second_hidden.forward_pass(ReLU_activation.outputs)

Sigmoid_activation.forward_pass(second_hidden.outputs)

loss = BinaryCrossEntropyloss.calculate(Sigmoid_activation.outputs, y_val)

#compare outputs from softmax to y for accuracy
probabilities =  Sigmoid_activation.outputs
predictions = np.round(probabilities)

accuracy = np.mean(predictions == y_val)

print(f'validation, acc: {accuracy:.3f},  loss: {loss:.3f}')



y_test = y_test.reshape(-1,1) 

first_hidden.forward_pass(df6_test_scaled)

ReLU_activation.forward_pass(first_hidden.outputs)

second_hidden.forward_pass(ReLU_activation.outputs)

Sigmoid_activation.forward_pass(second_hidden.outputs)

loss = BinaryCrossEntropyloss.calculate(Sigmoid_activation.outputs, y_test)

#compare outputs from softmax to y for accuracy
probabilities =  Sigmoid_activation.outputs
predictions = np.round(probabilities)

accuracy = np.mean(predictions == y_test)

print(f'test, acc: {accuracy:.3f},  loss: {loss:.3f}')







epoch: 0, acc: 0.550, loss: 0.693, data_loss: 0.693, reg_loss: 0.000, lr: 0.0003
epoch: 5, acc: 0.548, loss: 0.690, data_loss: 0.690, reg_loss: 0.000, lr: 0.00029998800047998075
epoch: 10, acc: 0.548, loss: 0.688, data_loss: 0.688, reg_loss: 0.000, lr: 0.0002999730024297813
epoch: 15, acc: 0.548, loss: 0.685, data_loss: 0.685, reg_loss: 0.000, lr: 0.0002999580058791769
epoch: 20, acc: 0.548, loss: 0.681, data_loss: 0.681, reg_loss: 0.000, lr: 0.00029994301082794265
epoch: 25, acc: 0.548, loss: 0.677, data_loss: 0.677, reg_loss: 0.001, lr: 0.00029992801727585374
epoch: 30, acc: 0.548, loss: 0.673, data_loss: 0.673, reg_loss: 0.001, lr: 0.0002999130252226854
epoch: 35, acc: 0.550, loss: 0.668, data_loss: 0.668, reg_loss: 0.001, lr: 0.0002998980346682128
epoch: 40, acc: 0.554, loss: 0.662, data_loss: 0.662, reg_loss: 0.001, lr: 0.00029988304561221125
epoch: 45, acc: 0.559, loss: 0.657, data_loss: 0.657, reg_loss: 0.001, lr: 0.000299868058054456
epoch: 50, acc: 0.562, loss: 0.650, data_los